<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/MutationandCrossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the Perpetrator's gender using an evolutionary algorithm to select features.

## - Mutation and Crossover

This code has all steps for the Mutation and Crossover. Where:

* Imports: the necessary libraries and the population; 
* Scripts: steps for do mutation or crossover;
* Setups: results.


## Imports

In [2]:
import pickle
import pandas as pd
import numpy as np
import random

In [3]:
with open("/content/drive/My Drive/Data Files/fitness.txt", "rb") as fp:   
  generation = pickle.load(fp)

## Scripts

In [4]:
def mutation_and_crossover(population):
  children = []
  prop_mutation = 0.1 #prob of mutation
  rand = random.uniform(0,1) #random value to compare with prob_mutation

  pop = [i[1] for i in generation]

  for p in population:
    parent1 = random.choice(pop) 
    parent2 = random.choice(pop)

    n = random.choice(range(1,len(parent1))) #gene that gona be mutaded
    
    if rand < prop_mutation:
      #mutation
      parent1[0] = random.choice(range(4)) #mute the model
      
      if parent1[n] == True:
        parent1[n] = False 
        child = parent1
      else:
        parent1[n] = True 
        child = parent1
      
      children.append(child)
    
    else:
      #crossover
      chromosome_length = len(parent1) #length of chromosome
      crossover_point = random.randint(1,chromosome_length-1) #point of crossover
      child1 = parent1[1:crossover_point] + parent2[crossover_point:]
      child2 = parent2[1:crossover_point] + parent1[crossover_point:]

      child1.insert(0,parent2[0]) #model+child1
      child2.insert(0,parent1[0]) #model+child2

      #print(child1)
      children.append(child1)
      children.append(child2)
  
  return children      

## Setups

In [5]:
run = mutation_and_crossover(generation)

In [7]:
pd.DataFrame(run)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,False,True,True,True,False,False,True,False,False,False,True,False,True,False,False,True,True
1,0,True,False,False,True,False,False,True,True,False,True,False,True,True,True,False,False,True
2,0,False,False,True,False,True,False,False,True,False,False,False,True,True,False,True,True,False
3,3,True,False,False,True,False,False,True,False,False,True,True,False,True,False,False,False,False
4,2,False,False,False,True,False,True,False,False,True,True,True,True,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,True,False,True,True,True,True,False,True,True,True,False,False,True,True,False,False,True
96,2,True,True,True,True,False,False,False,True,False,False,False,False,True,False,False,True,True
97,0,False,False,True,True,True,False,False,False,True,True,False,False,False,False,False,False,True
98,2,True,True,True,False,True,False,False,False,True,False,True,True,False,False,False,True,True


In [6]:
with open("/content/drive/My Drive/Data Files/mutandcross.txt", "wb") as mec: 
  pickle.dump(run, mec)